# Retrieval

Although the HTTP method suffices for our purposes, we shall present, for reasons of completeness, also an SSH variant for downloading the airport data that would be requisite should the csv file reside in a repository where client authentication via SSH is mandatory.

The following modules are required:

In [ ]:
import os
import subprocess
import shutil
import requests
import tempfile

from ipynb_utils import CFG

Moreover, we employ the following variables:

In [ ]:
# SSH address of the (not necessarily public) repository.
SSH_REPO_URL = "git@github.com:davidmegginson/ourairports-data.git"

# Path of the csv file relative to the (remote) repository root.
CSV_PATH_REL = "airports.csv"

# Target path of the csv file on the local machine.
AIRPORTS_DATA_PATH = CFG["AIRPORTS_DATA_PATH"]

# Full HTTP address of the csv file.
http_url = f"{HTTP_REPO_URL}/blob/main/{CSV_PATH_REL}"

The actual download is effected by the subsequent code cell.

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    subprocess.run(
        [
            "git",
            "clone",
            # Clones shallowly.
            "--depth",
            "1",
            # Skips blobs initially.
            "--filter=blob:none",
            # Enables sparse checkout mode.
            "--sparse",
            SSH_REPO_URL,
            tmpdir,
        ],
        check=True,
    )

    subprocess.run(
        [
            "git",
            "-C",
            tmpdir,
            # Initialises sparse checkout.
            "sparse-checkout",
            "init",
        ],
        check=True,
    )

    subprocess.run(
        [
            "git",
            "-C",
            tmpdir,
            # Specifies which files to include in the sparse checkout.
            "sparse-checkout",
            "set",
            CSV_PATH_REL,
            # Relaxes the checks as sparse-checkout expects directories.
            "--skip-checks",
        ],
        check=True,
    )

    src_file = os.path.join(tmpdir, CSV_PATH_REL)
    shutil.copy2(src_file, AIRPORTS_DATA_PATH)